In [ ]:
# problem 1 - toilet problem 
# problem 2 - voilence problem
# problem 3 - train coaches problem 
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Load datasets
pnr_data = pd.read_csv('SIH/pnr_numbers.csv')
tt_data = pd.read_csv('SIH/tte_personnel.csv')
station_master_data = pd.read_csv('SIH/station_masters.csv')
crpf_data = pd.read_csv('SIH/crpf_personnel.csv')
route_incharge_data = pd.read_csv('SIH/route_incharges.csv')

# Web portal configuration
PORTAL_URL = 'https://example-portal.com'
PORTAL_LOGIN_ID_FIELD = 'login_field_id'
PORTAL_PASSWORD_FIELD = 'password_field_id'
PORTAL_LOGIN_BUTTON = 'login_button_id'
PORTAL_MESSAGE_FIELD = 'message_field_id'
PORTAL_SEND_BUTTON = 'send_button_id'

def classify_problem(user_message):
    # Updated placeholder for AI model
    if 'railway' in user_message.lower() and 'coach' in user_message.lower():
        return 'problem_3'
    elif 'railway' in user_message.lower():
        return 'problem_1'
    elif 'security' in user_message.lower():
        return 'problem_2'
    else:
        return 'unknown'

def extract_pnr(user_message):
    match = re.search(r'\b\d{10}\b', user_message)
    if match:
        return match.group(0)
    return None

def get_route_from_pnr(pnr_number):
    route = pnr_data[pnr_data['pnr'] == pnr_number]['route']
    if not route.empty:
        return route.values[0]
    return None

def get_employee_details(route, employee_type):
    if employee_type == 'tt':
        return tt_data[tt_data['route'] == route]
    elif employee_type == 'station_master':
        return station_master_data[station_master_data['route'] == route]
    elif employee_type == 'crpf':
        return crpf_data
    elif employee_type == 'route_incharge':
        return route_incharge_data[route_incharge_data['route'] == route]
    else:
        return pd.DataFrame()

def send_message_to_portal(login_id, password, message):
    options = Options()
    options.add_argument("--headless")  # Run headless Chrome if needed
    driver = webdriver.Chrome(service=Service('path/to/chromedriver'), options=options)
    
    try:
        driver.get(PORTAL_URL)
        
        # Log in
        driver.find_element(By.ID, PORTAL_LOGIN_ID_FIELD).send_keys(login_id)
        driver.find_element(By.ID, PORTAL_PASSWORD_FIELD).send_keys(password)
        driver.find_element(By.ID, PORTAL_LOGIN_BUTTON).click()
        
        # Send the message
        driver.find_element(By.ID, PORTAL_MESSAGE_FIELD).send_keys(message)
        driver.find_element(By.ID, PORTAL_SEND_BUTTON).click()
    finally:
        driver.quit()

def forward_message(user_message):
    problem_type = classify_problem(user_message)
    pnr_number = extract_pnr(user_message)
    
    if pnr_number is None:
        print('PNR number not found in the message.')
        return

    route = get_route_from_pnr(pnr_number)
    if route is None:
        print(f'No route found for PNR {pnr_number}.')
        return

    if problem_type == 'problem_1':
        # Send message to TT, Station Master, and Route Incharge
        for _, tt in get_employee_details(route, 'tt').iterrows():
            send_message_to_portal(tt['tt_id'], tt['tt_password'], f'New railway issue reported: {user_message} with PNR: {pnr_number}')
        
        for _, station_master in get_employee_details(route, 'station_master').iterrows():
            send_message_to_portal(station_master['station_master_id'], station_master['station_master_password'], f'New railway issue reported: {user_message} with PNR: {pnr_number}')
        
        for _, route_incharge in get_employee_details(route, 'route_incharge').iterrows():
            send_message_to_portal(route_incharge['id'], route_incharge['password'], f'New railway issue reported: {user_message} with PNR: {pnr_number}')
        
    elif problem_type == 'problem_2':
        # Send message to CRPF
        for _, crpf in get_employee_details(route, 'crpf').iterrows():
            send_message_to_portal(crpf['id'], crpf['password'], f'New security issue reported: {user_message} with PNR: {pnr_number}')
    
    elif problem_type == 'problem_3':
        # Send message to TT, Station Master, and Route Incharge
        for _, tt in get_employee_details(route, 'tt').iterrows():
            send_message_to_portal(tt['tt_id'], tt['tt_password'], f'New coach issue reported: {user_message} with PNR: {pnr_number}')
        
        for _, station_master in get_employee_details(route, 'station_master').iterrows():
            send_message_to_portal(station_master['station_master_id'], station_master['station_master_password'], f'New coach issue reported: {user_message} with PNR: {pnr_number}')
        
        for _, route_incharge in get_employee_details(route, 'route_incharge').iterrows():
            send_message_to_portal(route_incharge['id'], route_incharge['password'], f'New coach issue reported: {user_message} with PNR: {pnr_number}')
    
    else:
        print('Unknown problem type.')

# Example usage
user_message = "There is a coach issue with PNR 1234567890."
forward_message(user_message)
